In [41]:
from tqdm.auto import tqdm
from osgeo import gdal, ogr
import geopandas as gpd
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
from pyproj import Proj, Transformer
import rasterio

In [29]:
os.environ['PROJ_LIB'] ='C:/Users/EBDL/anaconda3/Library/share/proj'

In [27]:
def crop_image_using_coordinates(InputImage, OutputImage, RefImage, latitude, longitude, CropSize):
    """extract tiff height and width"""
    Image = gdal.Open(RefImage, gdal.GA_ReadOnly)
    width = Image.RasterXSize
    height = Image.RasterYSize
    Image = None

    """extract vertex coordinates"""
    rds = rasterio.open(RefImage)
    rds.bounds
    left = rds.bounds[0]
    right = rds.bounds[2]
    top = rds.bounds[3]
    bottom = rds.bounds[1]

    """Convert coordinates system"""
    resolution_x = (right - left) / width
    resolution_y = (top - bottom) / height

    InputCrs = 'EPSG:4326'
    OutputCrs ='EPSG:4326'
    transformer = Transformer.from_crs(InputCrs, OutputCrs)
    longitude, latitude = transformer.transform(longitude, latitude)
    
    left_box = latitude - (resolution_x * CropSize)
    top_box = longitude + (resolution_y * CropSize)
    right_box = latitude + (resolution_x * CropSize)
    bottom_box = longitude - (resolution_y * CropSize)
    window = (left_box, top_box, right_box, bottom_box)

    gdal.Translate(OutputImage, InputImage, projWin = window)

# TRAIN DATA

## Crop NDVI

In [43]:
data=pd.read_csv("data/Raw_data/NDVI/data_concat.csv",encoding='cp949',index_col='Unnamed: 0')
data.drop('index',axis=1,inplace=True)
data

,발생일,발생시간,lon,lat,날씨,온도,습도,시간단위풍향,시간단위풍속,특보내용,target
0,20110122,233500,128.869839,37.782030,맑음,-1.000000,30.000000,45.0,8.000000,NaN,1
1,20110201,210700,128.852520,37.792612,맑음,1.000000,10.000000,225.0,8.000000,건조경보,1
2,20110226,164200,127.955332,37.379769,맑음,1.000000,550.000000,135.0,4.000000,NaN,1
3,20110212,61800,127.879749,37.220708,구름많음,-2.000000,29.000000,225.0,4.000000,NaN,1
4,20110211,175000,128.128864,37.772593,구름많음,-6.000000,44.000000,225.0,4.000000,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...
2140,20211115,없음,127.923819,37.883521,없음,2.146356,94.053007,없음,0.277835,없음,0
2141,20190217,없음,127.662696,37.974795,없음,-8.746485,85.655821,없음,0.525876,없음,0
2142,20140217,없음,127.798934,38.134524,없음,1.323292,63.784203,없음,2.931593,없음,0
2143,20140225,없음,127.713785,38.106001,없음,-1.105872,90.089028,없음,0.578714,없음,0


In [44]:
data['월']=[str(i) [4:6] for i in data['발생일']]
data['월']=data['월'].astype(int)

In [45]:
data['연도']=[str(i) [0:4] for i in data['발생일']]
data['연도']=data['연도'].astype(int)

In [46]:
RefImage = 'data/boundary/boundary_blank_resized.tif'
for i in tqdm(range(len(data))):
    
    year=data['연도'][i]
    
    if((data['월'][i]>=3) and (data['월'][i]<=5)):
        
        InputImage = f'NDVI/Color_Result/{year}-4.tif'
        
    elif((data['월'][i]>=6) and (data['월'][i]<=9)):
        
        InputImage = f'NDVI/Color_Result/{year}-8.tif'
        
    elif((data['월'][i]>=10) and (data['월'][i]<=11)):
        
        InputImage = f'NDVI/Color_Result/{year}-11.tif'
        
    else:
        
        InputImage = f'NDVI/Color_Result/{year}-1.tif'
    
    OutputImage = f'Final_data/NDVI/Crop_NDVI_{i}.tif'
    lon=data['lon'][i]
    lat=data['lat'][i]
    crop_image_using_coordinates(InputImage, OutputImage, RefImage, lon, lat, 1)

  0%|          | 0/2145 [00:00<?, ?it/s]

AttributeError: 'NoneType' object has no attribute 'RasterXSize'

AttributeError: 'NoneType' object has no attribute 'RasterXSize'
- 경로가 달라서 발생한 에러

## Crop 임상도

In [26]:
RefImage = 'data/boundary/boundary_blank_resized.tif'
for i in tqdm(range(len(data))):
           
    InputImage = '임상도/강원도_임상도_최종.tif'   
    OutputImage = f'Final_data/임상도/Crop_임상도_{i}.tif'
    
    lon=data['lon'][i]
    lat=data['lat'][i]
    
    crop_image_using_coordinates(InputImage, OutputImage, RefImage, lon, lat, 1)

  0%|          | 0/2145 [00:00<?, ?it/s]

SystemError: <built-in function Open> returned a result with an error set

## Crop 경사도

In [11]:
RefImage = 'data/boundary/boundary_blank_resized.tif'
for i in tqdm(range(len(data))):
           
    InputImage = '경사도/Slope_Final.tif'   
    OutputImage = f'Final_data/경사도/Crop_Slope_{i}.tif'
    
    lon=data['lon'][i]
    lat=data['lat'][i]
    
    crop_image_using_coordinates(InputImage, OutputImage, RefImage, lon, lat, 1)

  0%|          | 0/2145 [00:00<?, ?it/s]

## Crop 고도

In [12]:
RefImage = 'data/boundary/boundary_blank_resized.tif'
for i in tqdm(range(len(data))):
           
    InputImage = '고도/Height_Final.tif'   
    OutputImage = f'Final_data/고도/Crop_Height_{i}.tif'
    
    lon=data['lon'][i]
    lat=data['lat'][i]
    
    crop_image_using_coordinates(InputImage, OutputImage, RefImage, lon, lat, 1)

  0%|          | 0/2145 [00:00<?, ?it/s]

In [24]:
test='임상도/강원도_임상도_최종.tif'
test=rasterio.open(test)
image=pd.DataFrame(test.read(1))
image.to_csv("test.csv",encoding='utf-8-sig')

## Crop Landuse

In [39]:
%pwd

'C:\\Users\\EBDL\\Desktop\\Land'

In [47]:
data

,발생일,발생시간,lon,lat,날씨,온도,습도,시간단위풍향,시간단위풍속,특보내용,target,월,연도
0,20110122,233500,128.869839,37.782030,맑음,-1.000000,30.000000,45.0,8.000000,NaN,1,1,2011
1,20110201,210700,128.852520,37.792612,맑음,1.000000,10.000000,225.0,8.000000,건조경보,1,2,2011
2,20110226,164200,127.955332,37.379769,맑음,1.000000,550.000000,135.0,4.000000,NaN,1,2,2011
3,20110212,61800,127.879749,37.220708,구름많음,-2.000000,29.000000,225.0,4.000000,NaN,1,2,2011
4,20110211,175000,128.128864,37.772593,구름많음,-6.000000,44.000000,225.0,4.000000,NaN,1,2,2011
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2140,20211115,없음,127.923819,37.883521,없음,2.146356,94.053007,없음,0.277835,없음,0,11,2021
2141,20190217,없음,127.662696,37.974795,없음,-8.746485,85.655821,없음,0.525876,없음,0,2,2019
2142,20140217,없음,127.798934,38.134524,없음,1.323292,63.784203,없음,2.931593,없음,0,2,2014
2143,20140225,없음,127.713785,38.106001,없음,-1.105872,90.089028,없음,0.578714,없음,0,2,2014


In [49]:
RefImage = 'data/boundary/boundary_blank_resized.tif'
for i in tqdm(range(len(data))):
           
    InputImage = 'data/Raw_data/LANDUSE/Landuse.tif'   
    OutputImage = f'retest/train/landuse/Crop_Landuse_{i}.tif'
    
    lon=data['lon'][i]
    lat=data['lat'][i]
    
    crop_image_using_coordinates(InputImage, OutputImage, RefImage, lon, lat, 1)

  0%|          | 0/2145 [00:00<?, ?it/s]

## Crop popularity density

In [50]:
RefImage = 'data/boundary/boundary_blank_resized.tif'
for i in tqdm(range(len(data))):
           
    InputImage = 'data/Raw_data/POPDEN/population_density.tif'   
    OutputImage = f'retest/train/popden/Crop_Popden_{i}.tif'
    
    lon=data['lon'][i]
    lat=data['lat'][i]
    
    crop_image_using_coordinates(InputImage, OutputImage, RefImage, lon, lat, 1)

  0%|          | 0/2145 [00:00<?, ?it/s]

.

.

# TEST DATA
- 강원도 전체 지형데이터 Crop

## 데이터 불러오기

In [32]:
Raw_path='data/Raw_data/'
RefImage = 'data/boundary/boundary_blank_resized.tif'
Slope=Raw_path+'Slope/Slope_Final.tif'
Height=Raw_path+'Height/Height_Final.tif'
Forest=Raw_path+'Forest/강원도_임상도_최종.tif'
NDVI=Raw_path+'NDVI/Color_Result/2022-4.tif'
Landuse=Raw_path+'LANDUSE/Landuse.tif'
Popdensity=Raw_path+'POPDEN/population_density.tif'

## 좌표설정

In [22]:
%pwd

'C:\\Users\\EBDL\\Desktop\\Land'

기상데이터의 경위도 시작지점과 끝지점을 지형과 맞춰줍니다

In [11]:
N = 38.61370931
E = 129.359995
S = 37.03353708
W = 127.0950376

In [12]:
width = E-W
height= N-S
width_len=width/399
height_len=height/277

In [13]:
width_num=[]
height_num=[]
for i in range(400):
    width_num.append(W+width_len*i)
for i in range(278):
    height_num.append(N-height_len*i)

In [17]:
width_num=[round(i,7) for i in width_num]

In [25]:
width_num

[127.0950376,
 127.1007142,
 127.1063908,
 127.1120674,
 127.1177439,
 127.1234205,
 127.1290971,
 127.1347737,
 127.1404503,
 127.1461269,
 127.1518034,
 127.15748,
 127.1631566,
 127.1688332,
 127.1745098,
 127.1801864,
 127.185863,
 127.1915395,
 127.1972161,
 127.2028927,
 127.2085693,
 127.2142459,
 127.2199225,
 127.2255991,
 127.2312756,
 127.2369522,
 127.2426288,
 127.2483054,
 127.253982,
 127.2596586,
 127.2653351,
 127.2710117,
 127.2766883,
 127.2823649,
 127.2880415,
 127.2937181,
 127.2993947,
 127.3050712,
 127.3107478,
 127.3164244,
 127.322101,
 127.3277776,
 127.3334542,
 127.3391308,
 127.3448073,
 127.3504839,
 127.3561605,
 127.3618371,
 127.3675137,
 127.3731903,
 127.3788668,
 127.3845434,
 127.39022,
 127.3958966,
 127.4015732,
 127.4072498,
 127.4129264,
 127.4186029,
 127.4242795,
 127.4299561,
 127.4356327,
 127.4413093,
 127.4469859,
 127.4526625,
 127.458339,
 127.4640156,
 127.4696922,
 127.4753688,
 127.4810454,
 127.486722,
 127.4923985,
 127.4980751,
 

In [19]:
height_num=[round(i,7) for i in height_num]

In [24]:
height_num

[38.6137093,
 38.6080047,
 38.6023001,
 38.5965955,
 38.5908909,
 38.5851863,
 38.5794818,
 38.5737772,
 38.5680726,
 38.562368,
 38.5566634,
 38.5509588,
 38.5452542,
 38.5395496,
 38.533845,
 38.5281404,
 38.5224358,
 38.5167312,
 38.5110266,
 38.505322,
 38.4996175,
 38.4939129,
 38.4882083,
 38.4825037,
 38.4767991,
 38.4710945,
 38.4653899,
 38.4596853,
 38.4539807,
 38.4482761,
 38.4425715,
 38.4368669,
 38.4311623,
 38.4254577,
 38.4197532,
 38.4140486,
 38.408344,
 38.4026394,
 38.3969348,
 38.3912302,
 38.3855256,
 38.379821,
 38.3741164,
 38.3684118,
 38.3627072,
 38.3570026,
 38.351298,
 38.3455934,
 38.3398889,
 38.3341843,
 38.3284797,
 38.3227751,
 38.3170705,
 38.3113659,
 38.3056613,
 38.2999567,
 38.2942521,
 38.2885475,
 38.2828429,
 38.2771383,
 38.2714337,
 38.2657291,
 38.2600246,
 38.25432,
 38.2486154,
 38.2429108,
 38.2372062,
 38.2315016,
 38.225797,
 38.2200924,
 38.2143878,
 38.2086832,
 38.2029786,
 38.197274,
 38.1915694,
 38.1858648,
 38.1801603,
 38.17445

## CROP

### NDVI

In [33]:
InputImage=NDVI
num=0
for i in range(278):
    lat=height_num[i]
    for j in range(400):
        lon= width_num[j]
        OutputImage=f'좌표맞춘지형/NDVI_2022-4/Crop_NDVI_{num}.tif'
        num=num+1
        
        crop_image_using_coordinates(InputImage, OutputImage, RefImage, lon, lat, 1)

### Slope

In [34]:
InputImage=Slope
num=0
for i in range(278):
    lat=height_num[i]
    for j in range(400):
        lon= width_num[j]
        OutputImage=f'좌표맞춘지형/Slope/Crop_Slope_{num}.tif'
        num=num+1
        
        crop_image_using_coordinates(InputImage, OutputImage, RefImage, lon, lat, 1)

### Height

In [35]:
InputImage=Height
num=0
for i in range(278):
    lat=height_num[i]
    for j in range(400):
        lon= width_num[j]
        OutputImage=f'좌표맞춘지형/Height/Crop_Height_{num}.tif'
        num=num+1
        
        crop_image_using_coordinates(InputImage, OutputImage, RefImage, lon, lat, 1)

### Forest

In [36]:
# InputImage=Forest
# num=0
# for i in range(278):
#     lat=height_num[i]
#     for j in range(400):
#         lon= width_num[j]
#         OutputImage=f'좌표맞춘지형/Forest/Crop_Forest_{num}.tif'
#         num=num+1
        
#         crop_image_using_coordinates(InputImage, OutputImage, RefImage, lon, lat, 1)

### Landuse

In [37]:
InputImage=Landuse
num=0
for i in range(278):
    lat=height_num[i]
    for j in range(400):
        lon= width_num[j]
        OutputImage=f'좌표맞춘지형/Landuse/Crop_Landuse_{num}.tif'
        num=num+1
        
        crop_image_using_coordinates(InputImage, OutputImage, RefImage, lon, lat, 1)

### Pop_density

In [38]:
InputImage=Popdensity
num=0
for i in range(278):
    lat=height_num[i]
    for j in range(400):
        lon= width_num[j]
        OutputImage=f'좌표맞춘지형/Pop/Crop_Pop_{num}.tif'
        num=num+1
        
        crop_image_using_coordinates(InputImage, OutputImage, RefImage, lon, lat, 1)

.

.

# tif 형식을 npy로 변환합니다
- 모델에 들어가는 형식은 npy이기 때문

## Test tif to npy

In [6]:
#Forest_files= natsort.natsorted(glob(f"C:/Users/EBDL/Desktop/Land/Train_data/Forest/*.tif"))
Height_files= natsort.natsorted(glob(f"C:/Users/EBDL/Desktop/Land/retest/Height/*.tif"))
NDVI_files= natsort.natsorted(glob(f"C:/Users/EBDL/Desktop/Land/retest/NDVI_2022-4/*.tif"))
Slope_files= natsort.natsorted(glob(f"C:/Users/EBDL/Desktop/Land/retest/Slope/*.tif"))
Landuse_files= natsort.natsorted(glob(f"C:/Users/EBDL/Desktop/Land/retest/Landuse/*.tif"))
Pop_files= natsort.natsorted(glob(f"C:/Users/EBDL/Desktop/Land/retest/Pop/*.tif"))

In [ ]:
tif_list=[]
for i in range(len(Slope_files)):
    tmp = cv2.imread(Height_files[i], cv2.IMREAD_COLOR)
    tmp = cv2.cvtColor(tmp, cv2.COLOR_BGR2RGB)
    tmp = cv2.resize(tmp, (48, 48))
    tmp = tmp / 255.0
    tif_list.append(tmp)

test=np.array(tif_list)
np.save('Height_test.npy',test)

In [ ]:
tif_list=[]
for i in range(len(Slope_files)):
    tmp = cv2.imread(NDVI_files[i], cv2.IMREAD_COLOR)
    tmp = cv2.cvtColor(tmp, cv2.COLOR_BGR2RGB)
    tmp = cv2.resize(tmp, (48, 48))
    tmp = tmp / 255.0
    tif_list.append(tmp)

test=np.array(tif_list)
np.save('NDVI_test.npy',test)

In [ ]:
tif_list=[]
for i in range(len(Slope_files)):
    tmp = cv2.imread(Slope_files[i], cv2.IMREAD_COLOR)
    tmp = cv2.cvtColor(tmp, cv2.COLOR_BGR2RGB)
    tmp = cv2.resize(tmp, (48, 48))
    tmp = tmp / 255.0
    tif_list.append(tmp)

test=np.array(tif_list)
np.save('Slope_test.npy',test)

In [ ]:
tif_list=[]
for i in range(len(Slope_files)):
    tmp = cv2.imread(Landuse_files[i], cv2.IMREAD_COLOR)
    tmp = cv2.cvtColor(tmp, cv2.COLOR_BGR2RGB)
    tmp = cv2.resize(tmp, (48, 48))
    tmp = tmp / 255.0
    tif_list.append(tmp)

test=np.array(tif_list)
np.save('Landuse_test.npy',test)

In [14]:
tif_list=[]
for i in range(len(Slope_files)):
    tmp = cv2.imread(Pop_files[i], cv2.IMREAD_COLOR)
    tmp = cv2.cvtColor(tmp, cv2.COLOR_BGR2RGB)
    tmp = cv2.resize(tmp, (48, 48))
    tmp = tmp / 255.0
    tif_list.append(tmp)

test=np.array(tif_list)
np.save('Pop_test.npy',test)

## Train tif to npy

In [19]:
Height_train= natsort.natsorted(glob(f"C:/Users/EBDL/Desktop/Land/retest/test/Height/*.tif"))
NDVI_train= natsort.natsorted(glob(f"C:/Users/EBDL/Desktop/Land/retest/test/NDVI_2022-4/*.tif"))
Slope_train= natsort.natsorted(glob(f"C:/Users/EBDL/Desktop/Land/retest/test/Slope/*.tif"))
Landuse_train= natsort.natsorted(glob(f"C:/Users/EBDL/Desktop/Land/retest/train/landuse/*.tif"))
Pop_train= natsort.natsorted(glob(f"C:/Users/EBDL/Desktop/Land/retest/train/popden/*.tif"))

In [16]:
tif_list=[]
for i in range(len(Pop_train)):
    tmp = cv2.imread(Pop_train[i], cv2.IMREAD_COLOR)
    tmp = cv2.cvtColor(tmp, cv2.COLOR_BGR2RGB)
    tmp = cv2.resize(tmp, (48, 48))
    tmp = tmp / 255.0
    tif_list.append(tmp)

test=np.array(tif_list)
np.save('Pop_train.npy',test)

In [17]:
tif_list=[]
for i in range(len(Pop_train)):
    tmp = cv2.imread(Landuse_train[i], cv2.IMREAD_COLOR)
    tmp = cv2.cvtColor(tmp, cv2.COLOR_BGR2RGB)
    tmp = cv2.resize(tmp, (48, 48))
    tmp = tmp / 255.0
    tif_list.append(tmp)

test=np.array(tif_list)
np.save('Landuse_train.npy',test)

In [20]:
tif_list=[]
for i in range(len(Pop_train)):
    tmp = cv2.imread(Height_train[i], cv2.IMREAD_COLOR)
    tmp = cv2.cvtColor(tmp, cv2.COLOR_BGR2RGB)
    tmp = cv2.resize(tmp, (48, 48))
    tmp = tmp / 255.0
    tif_list.append(tmp)

test=np.array(tif_list)
np.save('Height_train.npy',test)

In [21]:
tif_list=[]
for i in range(len(Pop_train)):
    tmp = cv2.imread(NDVI_train[i], cv2.IMREAD_COLOR)
    tmp = cv2.cvtColor(tmp, cv2.COLOR_BGR2RGB)
    tmp = cv2.resize(tmp, (48, 48))
    tmp = tmp / 255.0
    tif_list.append(tmp)

test=np.array(tif_list)
np.save('NDVI_train.npy',test)

In [22]:
tif_list=[]
for i in range(len(Pop_train)):
    tmp = cv2.imread(Slope_train[i], cv2.IMREAD_COLOR)
    tmp = cv2.cvtColor(tmp, cv2.COLOR_BGR2RGB)
    tmp = cv2.resize(tmp, (48, 48))
    tmp = tmp / 255.0
    tif_list.append(tmp)

test=np.array(tif_list)
np.save('Slope_train.npy',test)